In [1]:
import torch
from torch import nn
from torch import optim
from torch.nn import Linear,Conv2d,MaxPool2d,Flatten,ReLU
import torch.functional as F
import torchvision

from torchvision import datasets
from torch.utils.data import DataLoader


In [2]:
# 数据准备
trans = torchvision.transforms.ToTensor
train_data = datasets.CIFAR10("./dataset/", True, transform=trans,download=True)
test_data = datasets.CIFAR10("./dataset/", False, transform=trans,download=True)

train_loader = DataLoader(train_data, 64, True, drop_last=False)
test_loader = DataLoader(test_data, 64, True, drop_last=False)

Files already downloaded and verified
Files already downloaded and verified


In [12]:
# 模型
class Module(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2, padding_mode="zeros"),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, ceil_mode=True),
            nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2, padding_mode="zeros"),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, ceil_mode=True),
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2, padding_mode="zeros"),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, ceil_mode=True),
            
            nn.Flatten(), # 64*4*4
            nn.Linear(64*4*4,64),
            nn.ReLU(),
            nn.Linear(64,10),
        )
    def forward(self,x):
        return self.model(x)
    
module =Module()
test_data = torch.ones((64,3,32,32)) # 输入的shape
out = module(test_data)
out.size() # 输出的shape - 64*10

torch.Size([64, 10])